In [ ]:
DATASETS_DIR = "C:/Users/mgonz/Desktop/yelp/"
DATASETS_DIR = "./"
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

from transformers import AutoTokenizer, AutoModel
import torch

from tqdm import tqdm

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
df_train = pd.read_csv(DATASETS_DIR + "train_reviews.csv")
df_test = pd.read_csv(DATASETS_DIR + "test_reviews.csv")
df_users = pd.read_csv(DATASETS_DIR + "usuarios.csv")
df_businesses = pd.read_csv(DATASETS_DIR + "negocios.csv")

### df_train

In [3]:
print("Train set shape:", df_train.shape)
df_train.head(5)

Train set shape: (967784, 9)


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,ZZO43qKB-s65zplC8RfJqw,-1BSu2dt_rOAqllw9ZDXtA,smkZq4G1AOm4V6p3id5sww,5.0,0,0,0,Fantastic fresh food. The greek salad is amazi...,2016-09-30 15:49:32
1,vojXOF_VOgvuKD95gCO8_Q,xpe178ng_gj5X6HgqtOing,96_c_7twb7hYRZ9HHrq01g,1.0,2,0,1,Been a patient at Largo Med/Diagnostic Clinic ...,2020-12-09 14:39:51
2,KwxdbiseRlIRNzpgvyjY0Q,axbaerf2Fk92OB4b9_peVA,e0AYjKfSF0DL-5C1CpOq6Q,4.0,0,0,0,The location is convenient to my campus so I d...,2013-09-04 16:19:51
3,3mwoBcTy-2gMh0L91uaIeA,_GOiybb0rImYKJfwyxEaGg,vF-uptiQ34pVLHJKzPHUlA,5.0,0,0,0,I agree with all the other compliments posted ...,2019-03-02 12:24:14
4,XfWf7XsBWs3kYyYq7Ns1ZQ,ojWKg3B5pH3ncAsxun3kUw,X28XK71RuEXPapeyUOwNzg,5.0,10,4,7,"Wanting to help out the local economy, I thoug...",2020-04-23 18:26:29


In [4]:
df_train["stars"].unique()

array([5., 1., 4., 2., 3.])

### df_test

In [5]:
print("Test set shape:", df_test.shape)
df_test.head(5)

Test set shape: (414765, 8)


,review_id,user_id,business_id,useful,funny,cool,text,date
0,ieYPmCImINjPzTDFmEKBKA,79F9QrQSet-b1yRCIM243Q,sXSUzImYOcRRI3xtG2M85g,1,0,1,Amazing coffee and chill atmosphere. The staff...,2018-01-29 04:33:28
1,QIkJ8fZ4yx_QaHahWWszAA,chuM6TBkFHtTwJ6z96Hj1A,Ipt9ga67vVC_2ob3YmVwNA,4,0,2,I pass by this joint every time I make a run t...,2011-01-10 03:10:49
2,seR2KhblYMWg-k9zzN6aYA,hF68a0mpu97u0oaryFYhyg,_RG4IByyBR528CMc7DefJA,2,0,0,Came here when my kitten got very sick by the ...,2015-09-06 15:29:02
3,BToo00Fi5pfJFA5MI2HM5g,G4yX5Q1tFfwSucFOmiyjdA,xxlbRiWWQkk-6LST3Hd12g,2,0,0,So I'll preface by saying we did have an overa...,2015-09-14 00:49:17
4,FHJAzi1imodBit3RWK7zQA,Srqi1xb7exdB9uRHxDeEkw,LgGqdFLD7-ca0Z9F_q4Fuw,0,0,0,This place is a joke. Worst bar service ever. ...,2015-07-24 01:03:40


### Creamos dataset de train y test del DF_TRAIN

In [6]:
df_x_train, df_x_val, y_train, y_val = train_test_split(df_train.drop(columns=["stars"]), df_train["stars"], test_size=0.2, random_state=42)

In [7]:
print("X_train shape:", df_x_train.shape)
print("X_val shape:", df_x_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (774227, 8)
X_val shape: (193557, 8)
y_train shape: (774227,)
y_val shape: (193557,)


In [8]:
df_x_train.head(5)

,review_id,user_id,business_id,useful,funny,cool,text,date
300516,VPhdXzdZOjQivhSbN_fwcQ,gqDbe1YwS4EhuCCNSNhpxg,Mzmt8AvZOExG_4h3MMlihg,0,0,0,"Love this place. Both times I came here, the c...",2017-11-25 04:47:11
695208,mjowKxIX4Yq_wHfLeSM4nA,CjlI1Jqs9RGtfLIaH4vvtQ,yRs15dgEYvxGatbDyeVXiA,1,0,0,Costa Vista is outstanding and flavorful food...,2020-10-13 03:15:17
921436,dny1-5ZqFZj8xTKlQZJQKA,Qjd7aIiKqnd68IfaR2EPoA,zXTO8GaLmRPATr505b4GQQ,7,1,8,I'm a big fan. A friend showed me a pic of his...,2018-09-29 01:33:27
397552,cde9T-0knSNa__eZWi3EFg,WikE3QVN3x5wpsagFvG06A,XdmaLMJE44rg75TYlOACVw,0,0,0,"Great team of PTs, the staff is friendly and k...",2015-01-26 16:22:48
953340,UCKlMaxkAMQXGoQOxtUQ8Q,WJR8f_QNG1kuPq5V6RxeFA,I2Yt74AzSQFfyQqgN7PSEg,5,1,3,Do not sleep on them. If you are looking for a...,2013-11-12 01:37:42


In [9]:
df_x_val.head(5)

,review_id,user_id,business_id,useful,funny,cool,text,date
862357,dpFUVfEksk_kM3knXJyKpg,QW6SlfQXFMPIM_ayiZRkhQ,mi63FnVXGeGgasWxUSeRsA,0,0,0,This was our second visit to Noble Crust and y...,2019-09-24 12:41:23
848879,0sazUYDnNHJWqKhbNhba9w,rD0LayQZv81BZkWcrK0dlw,8_I8oV3KPg5wKFYg5i3wdA,0,0,0,Called ahead to make sure we could be served. ...,2020-04-16 23:46:10
570332,UYGsjM8vhNkcJ6zAIiy-ng,xeK8j2kcxC9xL6eFQV5WWw,makMFAcU-TEeUPfJgdANZA,2,1,1,GO WOLFPACK!!!\n\nOk I am a fair-weather fan o...,2015-01-13 01:38:32
940147,_VHAe7-cnFf5LTbw5lkSpg,rceod_Lrve7bVTXsKfgIxQ,vIH8mShs03wkeIiW3qHUQg,0,0,0,I went here with my family and had pizza and t...,2015-10-24 14:41:55
317553,U4fyYCd4PKrcGjuIIxPV9A,sBTp3GPH2-YomCo-9-BWhw,TDKBPcViJQDMrdUm6a9XZA,3,0,2,We tried this place out for lunch the other da...,2015-01-27 04:24:14


### Pasamos los textos a embeddings BERT

In [ ]:
# ========== Carga del modelo ==========
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_encoder = AutoModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Dispositivo:", device)
model_encoder.to(device)
model_encoder.eval()

# ========== Pooling ==========
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / input_mask_expanded.sum(1)

# ========== Encoder ==========
def encode_texts(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model_encoder(**inputs)
            embeddings = mean_pooling(outputs, inputs['attention_mask'])
        all_embeddings.append(embeddings.cpu())
    return torch.cat(all_embeddings)

# ========== Generador de prompt individual ==========
def generar_prompt_individual(user_id, business_id, review_text, df_users, df_businesses):
    user_row = df_users[df_users["user_id"] == user_id]
    if user_row.empty:
        user_info = "User information not available."
    else:
        user_row = user_row.squeeze()
        review_count = user_row.get("review_count", "unknown")
        yelping_since = user_row.get("yelping_since", "unknown")
        year = str(yelping_since).split("-")[0] if pd.notna(yelping_since) else "unknown"
        average_stars = user_row.get("average_stars", "unknown")
        fans = user_row.get("fans", 0)
        elite = user_row.get("elite", "")
        elite_text = f"and has been elite in the years {elite}" if pd.notna(elite) and elite else "and has never been elite"
        fan_text = f"and has {fans} fans" if fans > 0 else "and has no fans"
        user_info = (
            f"The user has written {review_count} reviews since {year}, "
            f"has an average rating of {average_stars}, {fan_text}, {elite_text}."
        )

    business_row = df_businesses[df_businesses["business_id"] == business_id]
    if business_row.empty:
        business_info = "Business information not available."
    else:
        business_row = business_row.squeeze()
        business_name = business_row.get("name", "unknown")
        categories = business_row.get("categories", "unknown")
        stars = business_row.get("stars", "unknown")
        review_count_business = business_row.get("review_count", "unknown")
        is_open = business_row.get("is_open", 0)
        status = "open" if is_open == 1 else "closed"
        business_info = (
            f"About a business called {business_name}, categorized as {categories}, "
            f"with an average rating of {stars} and {review_count_business} reviews. "
            f"It is currently {status}."
        )

    review_part = f"They commented: '{review_text.strip()}'"
    prompt = f"{user_info} {review_part} {business_info}"
    return prompt

# ========== Procesamiento completo con bucles ==========


# -> TRAIN

print("📝 Generando prompts de entrenamiento...")
X_train_text = []
for _, row in tqdm(df_x_train.iterrows(), total=len(df_x_train), desc="Prompts entrenamiento"):
    prompt = generar_prompt_individual(row["user_id"], row["business_id"], row["text"], df_users, df_businesses)
    X_train_text.append(prompt)
    

    
# Guardar los prompts en un CSV
df_prompts_train = df_x_train.copy()
df_prompts_train["prompt"] = X_train_text

df_prompts_train[["user_id", "business_id", "text", "prompt"]].to_csv("prompts_entrenamiento.csv", index=False)

print("💾 Prompts de entrenamiento guardados en 'prompts_entrenamiento.csv'")



print("📐 Generando embeddings de entrenamiento...")
X_train = encode_texts(X_train_text)

print("💾 Guardando train en 'X_train.npy'")
np.save("X_train.npy", X_train)
np.save("y_train.npy", np.array(y_train))  # Guarda etiquetas alineadas




# -> VALIDACION

print("📝 Generando prompts de validación...")
X_val_text = []
for _, row in tqdm(df_x_val.iterrows(), total=len(df_x_val), desc="Prompts validación"):
    prompt = generar_prompt_individual(row["user_id"], row["business_id"], row["text"], df_users, df_businesses)
    X_val_text.append(prompt)
    
    
# Guardar los prompts de validación en un CSV
df_prompts_val = df_x_val.copy()
df_prompts_val["prompt"] = X_val_text

df_prompts_val[["user_id", "business_id", "text", "prompt"]].to_csv("prompts_validacion.csv", index=False)

print("💾 Prompts de validación guardados en 'prompts_validacion.csv'")




print("📐 Generando embeddings de validación...")
X_val = encode_texts(X_val_text)

print("💾 Guardando val en 'X_val.npy'")
np.save("X_val.npy", X_val)
np.save("y_val.npy", np.array(y_val))  # Guarda etiquetas alineadas




# -> TEST


print("📝 Generando prompts de test...")
X_test_text = []
for _, row in tqdm(df_test.iterrows(), total=len(df_test), desc="Prompts test"):
    prompt = generar_prompt_individual(row["user_id"], row["business_id"], row["text"], df_users, df_businesses)
    X_test_text.append(prompt)
    

    
# Guardar los prompts en un CSV
df_prompts_test = df_test.copy()
df_prompts_test["prompt"] = X_test_text

df_prompts_test[["user_id", "business_id", "text", "prompt"]].to_csv("prompts_test.csv", index=False)

print("💾 Prompts de test guardados en 'prompts_test.csv'")

print("📐 Generando embeddings de test...")
X_test = encode_texts(X_test_text)

print("💾 Guardando test en 'X_test.npy'")
np.save("X_test.npy", X_test)


🖥️ Dispositivo: cpu


### Entrenamos una red neuronal en keras

In [ ]:
import os

# Ruta al directorio donde están los .npy
base_path = r"./BERT_embeddings_dataset2"

# Cargar los datasets
X_train = np.load(os.path.join(base_path, "X_train.npy"))
y_train = np.load(os.path.join(base_path, "y_train.npy"))

X_val = np.load(os.path.join(base_path, "X_val.npy"))
y_val = np.load(os.path.join(base_path, "y_val.npy"))

X_test = np.load(os.path.join(base_path, "X_test.npy"))

print("✅ Todos los datasets fueron cargados correctamente.")

In [ ]:
input_dim = X_train.shape[1]
print("input_dim:", input_dim)

# Definir el modelo
model_tt = Sequential([
    Dense(256, activation='relu', input_shape=(input_dim,)),
    # Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    # Dropout(0.3),
    # Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Regresión
])

# Compilar el modelo
model_tt.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='mse',
    metrics=['mae']
)

# Entrenar
history =model_tt.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=64
)

model_tt.save("modelo_40.keras")


input_dim: 768


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  kernel_constraint: Constraint function applied to


Epoch 1/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 81s 7ms/step - loss: 1.0636 - mae: 0.7704 - val_loss: 0.6696 - val_mae: 0.6187
Epoch 2/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 80s 7ms/step - loss: 0.6456 - mae: 0.5875 - val_loss: 0.5702 - val_mae: 0.5368
Epoch 3/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 83s 7ms/step - loss: 0.5981 - mae: 0.5555 - val_loss: 0.5578 - val_mae: 0.5396
Epoch 4/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 80s 7ms/step - loss: 0.5750 - mae: 0.5376 - val_loss: 0.5551 - val_mae: 0.5169
Epoch 5/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 76s 6ms/step - loss: 0.5585 - mae: 0.5226 - val_loss: 0.5343 - val_mae: 0.5178
Epoch 6/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 82s 7ms/step - loss: 0.5481 - mae: 0.5143 - val_loss: 0.5386 - val_mae: 0.5030
Epoch 7/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 80s 7ms/step - loss: 0.5436 - mae: 0.5103 - val_loss: 0.5317 - val_mae: 0.5242
Epoch 8/40
12098/12098 ━━━━━━━━━━━━━━━━━━━━ 78s 6ms/step - loss: 0.5363 - mae: 0.5068 - val_loss: 0.5217 - val_mae: 0.4934
Epoch 9/40
12098

In [ ]:
# Predicción
y_pred = model_tt.predict(X_val)

# Métricas de regresión
rmse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("📊 Métricas de regresión:")
print(f" - RMSE: {rmse:.4f}")
print(f" - MAE: {mae:.4f}")
print(f" - R²: {r2:.4f}")

# Redondear predicciones antes de comparar
y_pred_rounded = np.round(y_pred).astype(int).astype(float)
acc = accuracy_score(y_val, y_pred_rounded)
print(f" - Accuracy (puntuaciones redondeadas): {acc:.4f}")

6049/6049 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step
📊 Métricas de regresión:
 - RMSE: 0.5048
 - MAE: 0.4988
 - R²: 0.7692
 - Accuracy (puntuaciones redondeadas): 0.6334


### Exportamos CSV con los resultados

In [ ]:
# Paso 1: Extraer los textos y los IDs
texts_test = df_test["text"].tolist()
review_ids = df_test["review_id"].tolist()

# Paso 2: Sacar embeddings (usando el mismo modelo que usaste para entrenamiento)
# X_test_embeddings = encode_texts(texts_test)
X_test_embeddings = X_test

# Paso 3: Predecir puntuaciones con XGBoost
y_pred_test = model_tt.predict(X_test_embeddings).flatten()  # Aseguramos que sea vector plano

# Paso 4: Crear DataFrame con las predicciones
df_predictions = pd.DataFrame({
    "review_id": review_ids,
    "stars": y_pred_test
})

# # Paso 5: Redondear si lo deseas (opcional)
# df_predictions["stars"] = df_predictions["stars"].round(1)
# Paso 5: Redondear a entero y forzar que termine en .0
df_predictions["stars"] = df_predictions["stars"].round().astype(int).astype(float)


# # Paso 6: Exportar a CSV
df_predictions.to_csv("predicciones_bert-base_keras-nn_prompts.csv", index=False)

print("✅ CSV generado con éxito.")

12962/12962 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step
✅ CSV generado con éxito.
